In [5]:
import pandas as pd

In [7]:
df = pd.read_csv("harlech_data_proportions.csv").dropna()

In [8]:
df.head()

,Unnamed: 0,Date,Year,Poverty,Population,Bottles Sold PC,Sales Volume PC,Volume Sold PC,Vodka Sold PC,Gin Sold PC,...,Pacific Prop,Two+ Prop,Other Prop,HighIncome Prop,LowIncome Prop,MidIncome Prop,Middle-Old Prop,Middle-Young Prop,Old Prop,Young Prop
0,0,2012-01-03,2012,10.6,7466.0,0.045406,0.572024,0.042743,0.005224,0.000268,...,0.000000,0.006464,0.001751,0.104878,0.262927,0.632195,0.290708,0.205457,0.391118,0.112716
1,1,2012-01-03,2012,10.2,3910.0,0.028900,0.381381,0.028885,0.004604,0.000767,...,0.000000,0.016444,0.000000,0.102477,0.222886,0.674637,0.308361,0.197993,0.366555,0.127090
2,2,2012-01-03,2012,9.7,5861.0,0.045897,0.496417,0.040203,0.012967,0.000000,...,0.000000,0.010223,0.000000,0.126021,0.324971,0.549008,0.305332,0.186828,0.402554,0.105287
3,3,2012-01-03,2012,9.6,25829.0,0.004181,0.049647,0.003659,0.001161,0.000000,...,0.000000,0.010386,0.002519,0.173219,0.200028,0.626754,0.327041,0.246467,0.307464,0.119027
4,4,2012-01-03,2012,15.7,131904.0,0.000970,0.010843,0.000835,0.000273,0.000000,...,0.000174,0.021593,0.015072,0.127840,0.284954,0.587206,0.253147,0.240095,0.287378,0.219379


In [10]:
predictive_df = df.drop([
    # Those we want to remove for all analyses
    'Unnamed: 0', 'Date',
    
    # Those we want to remove for the predictive model
    'Bottles Sold PC', 'Vodka Sold PC',
    'Gin Sold PC', 'Rum Sold PC', 'Whiskey Sold PC', 'Tequila Sold PC', 'Other Alc Sold PC',
    
    # Remove targets
    'Sales Volume PC','Volume Sold PC',
    
    # Remove linearly dependent columns
    'DOW', 'Young Prop', 'LowIncome Prop', 'Other Prop'], axis = 1)

In [ ]:
# Scale predictive data

In [ ]:
#### Create design matrix

```{r}
# Create X
X = as.matrix(predictive_scaled)

# Get number of rows
num_rows = dim(X)[1]

# Generate column of ones for the intercept
one_col = matrix(rep(1, num_rows), nrow = num_rows)

# Add column of 1s to left side of X
X = cbind(one_col, X)
```

***

```{r}
# Identify the number of parameters
num_param = dim(X)[2]

# Calculate hat matrix for OLS regression
OLS_hat_matrix = X %*% solve(t(X) %*% X) %*% t(X)

# Define lambda value
lambda = 1000

# Calculate matrix version of lambda (lambda * identity matrix)
lambda_matrix = lambda * diag(num_param)
```

```{r}
# Calculate hat matrix for RIDGE regression
RIDGE_hat_matrix = X %*% solve((t(X) %*% X) + lambda_matrix) %*% t(X)
```

```{r}
betas = solve((t(X) %*% X) + lambda_matrix) %*% t(X) %*% as.matrix(df$Rating)

data.frame(betas) %>% arrange(desc(betas)) %>% head(6)
```